In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../python/data/lfb_jan2013-mar2016 original.csv',sep=",")

c:\Users\Thang\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
oj = data[:5][["Easting_m","Northing_m"]]
oj["lat"] = 9
#print oj
eas = oj.get("Easting_m")
nor = oj.get("Northing_m")

#no null values in easting and northing
cleanedData = data[data["Easting_m"].notnull()][["Easting_m","Northing_m"]]


In [62]:
#convert easting northing to lat/lon
#might need to install BNG, -> pip install utm
import pyproj

bng = pyproj.Proj(init='epsg:27700')
wgs84 = pyproj.Proj(init='epsg:4326')

# AL1 1AB - pyproj.transform(from,to,easting,northing)
lon,lat = pyproj.transform(bng,wgs84, cleanedData["Easting_m"].values, cleanedData["Northing_m"].values)
cleanedData["lat"] = lat;
cleanedData["lon"] = lon;
#cleanedData

In [61]:
import json
import numpy as np

from sklearn.cluster import MiniBatchKMeans

#create new dataframe for output dataset
df = pd.DataFrame(data = cleanedData, columns=["lat","lon"])

input = df.values.tolist()
output = input

for i in range(2,7):
    print "k-" + str(i)
    mbk = MiniBatchKMeans(init='k-means++', n_clusters=i, batch_size=45,
                          n_init=10, max_no_improvement=10, verbose=0)
    mbk.fit(input)
    mbk_means_labels = mbk.labels_
    mbk_means_cluster_centers = mbk.cluster_centers_
    mbk_means_labels_unique = np.unique(mbk_means_labels)

    
    print mbk_means_cluster_centers

    cleanedData["kmeans"+str(i)] = mbk_means_labels
    

#for every k-# of a point there is a label(code) which indicates the cluster the point is in
#the code is used for visualization in D3 
df = pd.DataFrame(data = cleanedData, columns=["lat","lon","kmeans2","kmeans3","kmeans4","kmeans5","kmeans6"])

#export to json
df[::3].reset_index().to_json("cluster.json",orient='records')


k-2
[[  5.15045833e+01   4.71093733e-03]
 [  5.14975219e+01  -2.14137227e-01]]
k-3
[[ 51.5053299   -0.12299298]
 [ 51.5058915   -0.34336555]
 [ 51.52377608   0.06725369]]
k-4
[[ 51.5116293    0.08708305]
 [ 51.50733119  -0.34258377]
 [ 51.53935083  -0.114302  ]
 [ 51.41871873  -0.12112792]]
k-5
[[ 51.49335058  -0.22922312]
 [ 51.54124358  -0.09639724]
 [ 51.51699208   0.10596292]
 [ 51.4160444   -0.07506114]
 [ 51.51364582  -0.39459561]]
k-6
[[  5.13907953e+01  -1.12877981e-01]
 [  5.15186291e+01  -1.08035764e-01]
 [  5.15026271e+01   8.41976582e-02]
 [  5.15018373e+01  -3.73294554e-01]
 [  5.15183389e+01  -2.22719607e-01]
 [  5.15981924e+01  -4.35403401e-02]]


,lat,lon,kmeans2,kmeans3,kmeans4,kmeans5,kmeans6
0,51.549006,-0.192508,1,0,2,0,4
1,51.556094,-0.336755,1,1,1,4,3
2,51.493515,-0.237460,1,1,1,0,4
3,51.513620,-0.159464,1,0,2,1,1
4,51.509828,-0.080011,0,0,2,1,1
5,51.565753,-0.258720,1,1,1,0,4
6,51.541334,-0.198278,1,0,2,0,4
9,51.481386,-0.436728,1,1,1,4,3
15,51.513567,-0.087579,0,0,2,1,1
16,51.376437,-0.072162,0,0,3,3,0
